In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/heegyu.kim/Desktop/open-domain-dialog


In [2]:
from odd.gpt.gpt import GPTTask

In [3]:
task = GPTTask.load_from_checkpoint(
    "checkpoint/kodialogpt-base-v1.ckpt",
    )
task

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPTTask(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(51200, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (a

In [6]:
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    min_new_tokens=16,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

print(task.generate(["0 : 안녕하세요 \n 1 : 맙소사. 오늘 날씨 좋네요. 괜찮으세요? \n"], **generation_args)[0])

0 : 네 비가 오다말다 하네요. 비오면 추워지겠죠? 



In [7]:
task.model.save_pretrained("checkpoint/huggingface/kodialogpt-v1/")
task.tokenizer.save_pretrained("checkpoint/huggingface/kodialogpt-v1/")

('checkpoint/huggingface/kodialogpt-v1/tokenizer_config.json',
 'checkpoint/huggingface/kodialogpt-v1/special_tokens_map.json',
 'checkpoint/huggingface/kodialogpt-v1/vocab.json',
 'checkpoint/huggingface/kodialogpt-v1/merges.txt',
 'checkpoint/huggingface/kodialogpt-v1/added_tokens.json',
 'checkpoint/huggingface/kodialogpt-v1/tokenizer.json')

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [9]:
model = "huggingface/kodialogpt-v0/"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

[[{'generated_text': '0 : **는 게임 좋아하니\n1 : 나는 게임을 잘 안 해 키키 '}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 아이고... 진짜 무섭다... '}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 잘못 했길래 그래? '}]]

In [12]:
generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    min_new_tokens=32,
    do_sample=True,
    top_k=50,
    early_stopping=True
)
generator(
    ["0 : **는 게임 좋아하니\n1 :",
    "0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 :",
    "0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : "],
    **generation_args
)

[[{'generated_text': '0 : **는 게임 좋아하니\n1 : 난 게임을 잘 안 해 키키 '}],
 [{'generated_text': '0 : 어제 강남에서 살인사건 났대 ㅜㅜ 너무 무서워\n1 : 헐 왜? 무슨 일 있었어?\n0 : 사진보니까 막 피흘리는 사람있고 경찰들이 떠서 제압하고 난리도 아니었다던데??\n1 : 아 진짜? 근데 그건 좀 아니지 않아? '}],
 [{'generated_text': '0 : 자기야 어제는 나한테 왜 그랬어?\n1 : 뭔 일 있었어?\n0 : 어떻게 나한테 말도 없이 그럴 수 있어? 나 진짜 실망했어\n1 : 뭘 잘못 했길래 그래? '}]]